## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.model_selection import StratifiedKFold

In [3]:
cuda_use_gpus(0)

In [4]:
from keras import backend as K
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_lystdo_bi_lstm'

In [6]:
RANDOM_SEED = 42

In [7]:
np.random.seed(RANDOM_SEED)

## Read Data

In [8]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [9]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [10]:
y_train = load(features_data_folder + 'y_train.pickle')

In [11]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [12]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [13]:
def create_model():
    params = {
        'dense_dropout_rate': 0.07512852175097123,
        'lstm_dropout_rate': 0.3320541002991107,
        'num_dense': 130,
        'num_lstm': 333
    }
    
    embedding_layer = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )
    lstm_layer = Bidirectional(LSTM(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
    ))

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = lstm_layer(embedded_sequences_1)

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = lstm_layer(embedded_sequences_2)

    merged = concatenate([x1, y1])
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    merged = Dense(params['num_dense'], activation='relu')(merged)
    merged = Dropout(params['dense_dropout_rate'])(merged)
    merged = BatchNormalization()(merged)

    output = Dense(1, activation='sigmoid')(merged)

    model = Model(
        inputs=[sequence_1_input, sequence_2_input],
        outputs=output
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

In [14]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [15]:
NUM_FOLDS = 5
NUM_EPOCHS = 35

In [16]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [17]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [18]:
%%time

for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
    X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])

    X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
    X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])

    y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
    y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model()
    model.fit(
        [X_fold_train_q1, X_fold_train_q2], y_fold_train,
        validation_data=([X_fold_val_q1, X_fold_val_q2], y_fold_val),
#         class_weight=keras_get_class_weights(y_fold_val),

        batch_size=384,
        epochs=NUM_EPOCHS,
        verbose=1,

        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_pred_oofp = model.predict([X_train_q1[ix_val], X_train_q2[ix_val]], batch_size=2048, verbose=1).reshape(-1)
    y_pred_oofp += model.predict([X_train_q2[ix_val], X_train_q1[ix_val]], batch_size=2048, verbose=1).reshape(-1)
    y_pred_oofp /= 2
       
    # Remember them.
    y_train_oofp[ix_val] = y_pred_oofp
    
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_val_q1
    del X_fold_val_q2
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 646862 samples, validate on 161718 samples
Epoch 1/35
646862/646862 [==============================] - 266s - loss: 0.4940 - acc: 0.7528 - val_loss: 0.4293 - val_acc: 0.7923
Epoch 2/35
646862/646862 [==============================] - 269s - loss: 0.4117 - acc: 0.8031 - val_loss: 0.3903 - val_acc: 0.8167
Epoch 3/35
646862/646862 [==============================] - 268s - loss: 0.3763 - acc: 0.8235 - val_loss: 0.3696 - val_acc: 0.8287
Epoch 4/35
646862/646862 [==============================] - 267s - loss: 0.3513 - acc: 0.8374 - val_loss: 0.3611 - val_acc: 0.8335
Epoch 5/35
646862/646862 [==============================] - 267s - loss: 0.3327 - acc: 0.8477 - val_loss: 0.3641 - val_acc: 0.8321
Epoch 6/35
646862/646862 [==============================] - 269s - loss: 0.3190 - acc: 0.8551 - val_loss: 0.3544 - val_acc: 0.8407
Epoch 7/35
646862/646862 [==============================] - 268s - loss: 0.3062 - acc: 0.8617 - val_loss: 0.3686 - val_acc: 0.8346
Epoch 8/3

646864/646864 [==============================] - 269s - loss: 0.3201 - acc: 0.8546 - val_loss: 0.3568 - val_acc: 0.8403
Epoch 7/35
646864/646864 [==============================] - 267s - loss: 0.3077 - acc: 0.8613 - val_loss: 0.3650 - val_acc: 0.8357
Epoch 8/35
646864/646864 [==============================] - 269s - loss: 0.2975 - acc: 0.8662 - val_loss: 0.3636 - val_acc: 0.8372
Epoch 9/35
646864/646864 [==============================] - 269s - loss: 0.2905 - acc: 0.8701 - val_loss: 0.3673 - val_acc: 0.8404
Epoch 10/35
646864/646864 [==============================] - 271s - loss: 0.2828 - acc: 0.8742 - val_loss: 0.3684 - val_acc: 0.8403
Epoch 00009: early stopping
80858/80858 [==============================] - 8s     

Fitting fold 4 of 5

Train on 646866 samples, validate on 161714 samples
Epoch 1/35
646866/646866 [==============================] - 269s - loss: 0.4944 - acc: 0.7522 - val_loss: 0.4344 - val_acc: 0.7898
Epoch 2/35
646866/646866 [==============================] - 269s - 

646866/646866 [==============================] - 267s - loss: 0.3198 - acc: 0.8539 - val_loss: 0.3631 - val_acc: 0.8395
Epoch 7/35
646866/646866 [==============================] - 267s - loss: 0.3078 - acc: 0.8606 - val_loss: 0.3549 - val_acc: 0.8452
Epoch 8/35
646866/646866 [==============================] - 266s - loss: 0.2978 - acc: 0.8659 - val_loss: 0.3629 - val_acc: 0.8397
Epoch 9/35
646866/646866 [==============================] - 265s - loss: 0.2896 - acc: 0.8698 - val_loss: 0.3599 - val_acc: 0.8456
Epoch 10/35
646866/646866 [==============================] - 266s - loss: 0.2833 - acc: 0.8735 - val_loss: 0.3578 - val_acc: 0.8457
Epoch 11/35
646866/646866 [==============================] - 266s - loss: 0.2773 - acc: 0.8764 - val_loss: 0.3614 - val_acc: 0.8453
Epoch 00010: early stopping
80857/80857 [==============================] - 8s     
CPU times: user 4h 37min 43s, sys: 30min 29s, total: 5h 8min 12s
Wall time: 4h 3min 33s


## Save feature names

In [19]:
feature_names = [
    'oofp_lystdo_bi_lstm',
]

In [20]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save Train features

In [21]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [22]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [23]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [24]:
model = create_model()
model.load_weights(model_checkpoint_path)

In [25]:
# It would be better to fit the model on the whole training set
# but the validation set for early stopping would be an issue.
y_test_oofp = model.predict([X_test_q1, X_test_q2], batch_size=2048, verbose=1).reshape(-1)
y_test_oofp += model.predict([X_test_q2, X_test_q1], batch_size=2048, verbose=1).reshape(-1)
y_test_oofp /= 2

2345796/2345796 [==============================] - 237s   


In [26]:
y_test_oofp = y_test_oofp.reshape((-1, 1))

In [27]:
save(y_test_oofp, features_data_folder + f'X_test_{feature_list_id}.pickle')